# **Laboratorio 6**
## **Parte 2**

- Derek Arreaga - 22537
- Paula Barillas - 22764
- Mónica Salvatierra - 22249

### **Teoría** 

### **Parte 2**

Defina y responda

**1. La matriz Jacobiana es una "instantánea" de las fuerzas del sistema en un punto específico. Explique conceptualmente qué representa el elemento en la esquina inferior izquierda de la matriz (∂(dI/dt)/∂S). ¿Qué nos dice este término sobre la relación entre los susceptibles y la propagación de la infección?**

- El término (∂(dI/dt)/∂S) representa cómo va cambiando la velocidad de crecimiento de los infectados (dI/dt) cuando se modifica ligeramente la cantidad de personas de susceptibles (S), manteniendo todo lo demás constante. Es decir, indica cuánto influye el número de personas susceptibles en la propagación de la infección. Si este valor es positivo y grande, significa que un pequeño aumento en los susceptibles genera un incremento notable en el número de infecciones, mostrando que el sistema es muy sensible a los cambios en la población susceptible.

<br>

**2. El análisis muestra que cuando R₀ > 1, el Equilibrio Libre de Enfermedad es inestable. Describa en una o dos frases qué significa esto en el contexto de una política de salud pública de "esperar y ver". ¿Por qué la inestabilidad de este punto hace que esa política sea arriesgada?**

- Cuando R₀ > 1, el equilibrio libre de enfermedad es inestable, lo que significa que si aparece un pequeño número de infectados, la enfermedad tenderá a propagarse en lugar de desaparecer. En una política de salud pública de “esperar y ver”, esto implica un riesgo alto, porque un brote inicial mínimo puede ocasionar que los contagios crezcan rápidamente. Por eso, no intervenir desde el inicio puede permitir que la epidemia se descontrole antes de poder reaccionar.

<br>


**3. El Equilibrio Endémico, cuando existe, suele ser estable. ¿Qué significa "estabilidad" en este contexto? Si una nueva variante del virus, más contagiosa, causa un aumento repentino de casos (una perturbación), ¿qué predice la estabilidad de este equilibrio sobre el comportamiento del sistema a largo plazo?**

- La estabilidad del equilibrio endémico significa que, si el sistema se desvía ligeramente de ese punto, tenderá a regresar gradualmente a su estado estable. Si surge una variante más contagiosa que provoque un pico repentino de casos, la estabilidad predice que, con el tiempo, los contagios volverán a un nivel constante, aunque ese nuevo nivel podría ser más alto dependiendo del valor actualizado de R₀.

<br>

**4. En un modelo de ecuaciones, un equilibrio estable es un punto fijo. En un MBA, debido a la aleatoriedad (estocasticidad), el sistema nunca se asienta perfectamente en un punto. ¿Cómo se manifestaría un "equilibrio estable" en el espacio de estados proyectado de un MBA? Describa cómo se vería la trayectoria del sistema una vez que ha alcanzado este estado de equilibrio.**

- En un modelo basado en agentes, la estabilidad no se ve como un punto fijo exacto, ya que la aleatoriedad hace que los valores varíen constantemente. Un equilibrio estable se manifestaría como una región densa en el espacio de estados, donde el sistema oscila alrededor de un promedio sin desviarse mucho. La trayectoria del sistema mostraría cambios pequeños y continuos alrededor de ese punto de equilibrio, en lugar de converger a un valor único.

<br>

### **Referencias**

1. Equilibrio endémico. (s. f.). ScienceDirect. Recuperado 16 de octubre de 2025, de https://www.sciencedirect.com/topics/mathematics/endemic-equilibrium

2. Zhang, X., Wang, J., Yu, C., Fei, J., Luo, T., & Cao, Z. (2025). Agent-Based Modeling of Epidemics: Approaches, applications, and future directions. Technologies, 13(7), 272. https://doi.org/10.3390/technologies13070272
